In [3]:
# 🚀 Auto-setup: installs deps + configures CFBD access
%run ./_auto_setup.py


🔧 Notebook auto-setup complete:
  • Project root: /Users/stephen_bowman/Documents/GitHub/Script_Ohio_2.0
  • Requirements: /Users/stephen_bowman/Documents/GitHub/Script_Ohio_2.0/requirements.txt
  • Missing modules: none
  • Auto install performed: False


# 12 · Update Training Data

Use this notebook as a living runbook for refreshing `model_pack/updated_training_data.csv` whenever new CFBD data arrives. The workflow is now automated by `model_pack/scripts/build_training_dataset.py` and the `refresh-training` subcommand in `project_management/data_workflows.py`.

**Pipeline overview**

1. **Migrate raw starter-pack games** to the 81-feature schema (88 total columns) via `StarterPackDataMigrator`.
2. **Extend** `updated_training_data.csv` with the migrated rows (deduped on `game_key`).
3. **Retrain** models (optional) so notebooks/agents immediately consume the latest weights.

> 🔑 Requires `CFBD_API_KEY` plus the optional dependencies from `requirements-optional.txt` if you intend to regenerate features from scratch.


In [4]:
import shlex
import subprocess
import sys
from pathlib import Path

# Use python3 and correct path
# Get the model_pack directory (parent of this notebook's directory)
model_pack_dir = Path().resolve()
if model_pack_dir.name != "model_pack":
    # If running from different location, find model_pack
    model_pack_dir = Path(__file__).parent if '__file__' in globals() else Path.cwd()
    if model_pack_dir.name != "model_pack":
        model_pack_dir = model_pack_dir.parent / "model_pack"

script_path = model_pack_dir / "scripts" / "build_training_dataset.py"

command = [
    sys.executable,  # Use python3 from current environment
    str(script_path),
    "--max-week",
    "13",
    "--train-models",
]

print("Running:\n", " ".join(shlex.quote(part) for part in command))
result = subprocess.run(command, check=False, cwd=str(model_pack_dir))
print(f"\nExit code: {result.returncode}")



Running:
 /Users/stephen_bowman/Documents/GitHub/Script_Ohio_2.0/.venv/bin/python /Users/stephen_bowman/Documents/GitHub/Script_Ohio_2.0/model_pack/scripts/build_training_dataset.py --max-week 13 --train-models

Exit code: 1


Traceback (most recent call last):
  File "/Users/stephen_bowman/Documents/GitHub/Script_Ohio_2.0/model_pack/scripts/build_training_dataset.py", line 28, in <module>
    from project_management.data_workflows import FastAIModelTrainer, TrainingDataExtender
ModuleNotFoundError: No module named 'project_management.data_workflows'


## CLI reference

```
python project_management/data_workflows.py refresh-training \
  --max-week 13 \              # optional filter for in-progress seasons
  --skip-migration \           # if you already have a migrated CSV
  --migrated-file path.csv \   # custom migrated CSV when skipping migration
  --skip-extend \              # preview output without touching updated_training_data.csv
  --train-models \             # retrain FastAI neural model after extension
  --epochs 400 \
  --learning-rate 5e-4
```

The command internally calls `model_pack/scripts/build_training_dataset.py`, so you can run that script directly if you prefer to stay inside the model_pack directory.
